In [ ]:
import csv
import re
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
from string import punctuation
import json
from helpers import *

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
data_path = './../../../data'

expert_terms = get_exp_terms(data_path)
advocate_dir = os.path.join(data_path, 'BeerAdvocate')
reviews_ba = pd.read_csv(os.path.join(advocate_dir, 'ratings_BA_clean.csv'))

rev_true_ba = reviews_ba[reviews_ba['review']==True]

In [5]:
users = pd.read_csv(os.path.join(advocate_dir, 'users.csv'))
beer_BA = pd.read_csv(os.path.join(advocate_dir, 'beers.csv'))

In [6]:
beer_BA_grouped = beer_BA.groupby('style')
most_fam_b_per_style = pd.DataFrame()
for name, gruop in beer_BA_grouped:
    g = gruop.sort_values(ascending=[False],by=['nbr_reviews'])
    most_fam_b_per_style = pd.concat([most_fam_b_per_style, g.head(1)])

most_fam_stout = most_fam_b_per_style.sort_values(ascending=[False],by=['nbr_reviews']).iloc[1]
reviews_Stout = reviews_ba[reviews_ba['beer_id']==most_fam_stout.beer_id]
text_rev_stout = reviews_Stout[reviews_Stout['review']==True]['text']

In [7]:
expert_terms_stemmed = {}
for category, terms in expert_terms.items():
    tokens = [word_tokenize(term.lower()) for term in terms]
    
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(word[0]) for word in tokens]
    expert_terms_stemmed[category] =  stemmed_tokens



In [8]:
exp_stem_set = []
for category, terms in expert_terms_stemmed.items():
    exp_stem_set = exp_stem_set + terms
    
exp_stem_set = set(exp_stem_set)

In [14]:
def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punkt_symbols = set(punctuation)
    removable_words= stop_words.union(punkt_symbols)

    filtered_tokens = [word for word in tokens if ((word not in removable_words) and word.isalpha())]

    # Stemming
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return stemmed_tokens

In [15]:
def exp_stem_extract(text, exp_stem_set):
    extracted_keywords = set(extract_keywords(text))
    return list(exp_stem_set.intersection(extracted_keywords))

In [16]:
def stem_df(x, exp_stem_set):
    x = x.copy()
    x.loc[:, 'text_lower'] = x['text'].str.lower()
    stems =  x['text_lower'].apply(lambda x: exp_stem_extract(x,exp_stem_set))
    x = x.drop(columns=['text', 'text_lower'])
    return pd.concat([x, stems], axis=1).rename(columns={'text_lower':'stems'})

In [17]:
text_rev_stout = reviews_Stout[reviews_Stout['review']==True][['user_id', 'text']]


In [18]:
import time

start_time = time.time()

# Concatenate the results back with the original DataFrame
text_rev_stout = stem_df(text_rev_stout, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2.485391139984131 seconds ---


In [19]:
import time
start_time = time.time()

rev_true_ba = stem_df(rev_true_ba, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1597.5862839221954 seconds ---


In [21]:
rev_true_ba = rev_true_ba[rev_true_ba['lang_tag']=='en']
rev_true_ba.to_pickle(os.path.join(advocate_dir, 'reviews_with_exp_stems.pkl'))

In [22]:
len(rev_true_ba)

2588899

In [23]:
data_path = './../../../data'
rb_dir = os.path.join(data_path, 'RateBeer')
reviews_rb = pd.read_csv(os.path.join(rb_dir, 'ratings_RB_clean.csv'))

rev_true_rb = reviews_rb[reviews_rb['review']==True]

In [24]:
len(rev_true_rb)

7121601

In [25]:
rev_true_rb = rev_true_rb[rev_true_rb['lang_tag']=='en']

In [26]:
len(rev_true_rb)

6606948

In [ ]:
len(rev_true_rb)

6606948

In [27]:
import time 
start_time = time.time()

rev_true_rb = stem_df(rev_true_rb, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2582.77440738678 seconds ---


In [28]:
rev_true_rb.to_pickle(os.path.join(rb_dir, 'reviews_with_exp_stems.pkl'))